### The RIDB API
<img src="images/hobbit_house_expedia.jpeg" style="display: inline-block">
<br>
image source: expedia.com
<br>
<br>
For an RIDB API Key: https://ridb.recreation.gov/?action=register

In [1]:
import pandas as pd 
import numpy as np
import config
import requests
import json
from pandas.io.json import json_normalize

#### Get the data from RDIB

In [2]:
ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"

In [3]:
camping_params = params=dict(activity_id=9, apiKey = config.RIDB_API_KEY,\
                             latitude=45.4977712, longitude=-121.8211673, radius=15)
response = requests.get(ridb_facilities_url,camping_params)
camping_json  = json.loads(response.text)
camping_df = json_normalize(camping_json['RECDATA'])

In [4]:
camping_df.FacilityName

0                                RILEY HORSE CAMPGROUND
1                                           STILL CREEK
2                                              TOLLGATE
3                                            CAMP CREEK
4                                            LOST CREEK
5                                    TILLY JANE A-FRAME
6                              TILLY JANE GUARD STATION
7                              WILDWOOD RECREATION SITE
8                         EAGLE CREEK OVERLOOK GRP SITE
9                            Hood River Ranger District
10                              Castle Canyon Trailhead
11                                 Cool Creek Trailhead
12                             Dog River West Trailhead
13                                   Elk Cove Trailhead
14                               Herman Creek Trailhead
15                                Hidden Lake Trailhead
16                                Black Lake Campground
17                          Cloud Cap Saddle Cam

In [5]:
mock_url = "http://" + config.LAMP_IP + "/ridb_mock.json"
camping_df = pd.read_json(mock_url)

In [6]:
camping_df.head()

,FacilityAdaAccess,FacilityDescription,FacilityDirections,FacilityEmail,FacilityID,FacilityLatitude,FacilityLongitude,FacilityMapURL,FacilityName,FacilityPhone,FacilityReservationURL,FacilityTypeDescription,FacilityUseFeeDescription,Keywords,LastUpdatedDate,LegacyFacilityID,OrgFacilityID,StayLimit
0,,<h2>Overview</h2>Wildwood Recreation Site i...,Wildwood Recreation Site is located 39 m...,,234075,45.3561,-121.987,,WILDWOOD RECREATION SITE,503-622-3696,,Camping,,,2016-05-12,74082,AN374082,
1,,<p>This small rustic campground is locate...,"<p><u>From Prineville , Oregon</u>:</p><p>Tr...",,236929,44.4839,-120.337,,Wildwood Campground,,,,,,2016-05-09,,38780,
2,,<h2>Overview</h2>Whispering Falls Campground...,"8.3 miles southeast of Detroit , Oregon: ...",,251470,44.6878,-122.009,,WHISPERING FALLS CAMPGROUND,503-854-3366,,Camping,,,2016-05-12,127540,AN427540,
3,,<p>The Resort is situated on the shores ...,"<p><u>From Portland , OR</u> Lost Lake Re...",,235897,45.5008,-121.816,,Lost Lake Resort,,,,,,2016-05-09,,53230,
4,,<p>Lost Lake Campground is adjacent to H...,"<p>From McKenzie Bridge , OR , follow Hig...",,244288,44.4293,-121.912,,Lost Lake Campground,,,,,,2016-05-09,,13362,


In [7]:
camping_df.LastUpdatedDate.unique()

array(['2016-05-12', '2016-05-09', '2015-10-15', '2015-12-03'], dtype=object)

In [8]:
camping_df.LegacyFacilityID.unique()

array([74082.0, '', 127540.0, 125541.0, 71641.0, 71633.0, 75097.0, 71620.0,
       122890.0], dtype=object)

#### Clean up the data

In [9]:
camping_df.shape

(17, 18)

In [10]:
camping_df = camping_df.replace('', np.nan)

If using live data from RIDB, drop geojson coordinates - Lat/Long information duplicated, mysql 5.4 doesnt support geojson types (but 5.7 does!)

In [11]:
#camping_df = camping_df.drop(['GEOJSON.COORDINATES','GEOJSON.TYPE'], axis=1)

ValueError: labels ['GEOJSON.COORDINATES' 'GEOJSON.TYPE'] not contained in axis

For this tutorial, we only want campgrounds with Lat/Long data. Drop any campgrounds that dont have it.

In [ ]:
camping_df[camping_df.FacilityLatitude.isnull()]

In [ ]:
camping_df = camping_df.dropna(subset=['FacilityLatitude','FacilityLongitude'])

In [ ]:
camping_df[camping_df.FacilityLatitude.isnull()]

When working with data - always check shape!!

In [ ]:
camping_df.shape

#### Store the data

In [ ]:
camping_df.to_csv('test.csv', index=False)

In [ ]:
csv_test = pd.read_csv('test.csv')

In [ ]:
csv_test.head()

In [ ]:
from sqlalchemy import create_engine
connectStr = "mysql+pymysql://" + config.DB_USER + ":" + config.DB_PASS + "@" + config.DB_HOST +  "/" + config.DB_NAME
engine =create_engine(connectStr)

In [ ]:
camping_df.to_sql('test',engine,if_exists='replace')

In [ ]:
sql_test = pd.read_sql('select * from test', engine, index_col='index')

In [ ]:
sql_test.head()

In [ ]:
sql_test.LastUpdatedDate.unique()

In [ ]:
sum(sql_test.LegacyFacilityID.isnull())